In [ ]:
%pip install pybaseball==2.2.1
%matplotlib inline
import pandas as pd; 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import seaborn as sns
import sklearn as sk
import pybaseball as pyb
from matplotlib import pyplot as plt
from pybaseball import cache
cache.enable()
import warnings; warnings.filterwarnings('ignore')


In [ ]:
example_df = pd.DataFrame({
   # 'Team' : ['HOU', 'MIN', 'NYY', 'LAD', 'BOS', 'WSN', 'ATL', 'CHC', 'COL', 'OAK', 'NYM', 'MIL' 'TBR', 'ARI', 'CLE', 'TEX', 'LAA', 'PHI', 'STL', 'SEA', 'PIT', 'CIN', 'TOR', 'CHW', 'BAL', 'SDP', 'KCR', 'SFG', 'DET','MIA'],
    '2018_hr_total' : [208, 267, 235, 216, 227, 205, 191, 210, 150, 218, 167, 175, 217, 172, 205, 194, 214, 166, 176, 157, 186, 176, 170, 182, 155, 188, 135, 162, 133, 128],
    '2018_ab_total' : [5623, 5515, 5572, 5595, 5579, 5453, 5517, 5541, 5475, 5542, 5624, 5582, 5477, 5532, 5498, 5453, 5472, 5526, 5513, 5447, 5424, 5460, 5468, 5523, 5505, 5523, 5505, 5507, 5494, 5486,],
    '2019_hr_total' : [288, 307, 306, 279, 245, 231, 249, 256, 224, 257, 242, 250, 217, 220, 223, 223, 220, 215, 210, 239, 163, 227, 247, 182, 213, 219, 162, 167, 149, 146],
    '2019_ab_total' : [5613, 5732, 5583, 5493, 5770, 5512, 5560, 5461, 5660, 5561, 5624, 5542, 5628, 5633, 5425, 5540, 5542, 5571, 5449, 5500, 5657, 5450, 5493, 5529, 5596, 5391, 5496, 5579, 5549, 5512, ]


})

In [ ]:
example_df

In [ ]:
hits_df = pyb.team_batting(2015, 2019)
hits_2019 = pyb.team_batting(2018, 2019)

In [ ]:
# creating our df for predicting a full 2020 season 

hits_df_copy = hits_df.copy()

hits_df_copy = hits_df_copy.loc[:, ['Season', 'Team', 'G', 'AB', 'H', 'HR', 'RBI', 'AVG', 'OBP', 'SLG', 'OPS', 'wRC', 'WAR']]

hits_df_copy['G_Next_Year'] = hits_df_copy.sort_values(['Season', 'Team'], ascending=False).groupby('Team')['G'].shift()
hits_df_copy['H_Next_Year'] = hits_df_copy.sort_values(['Season', 'Team'], ascending=False).groupby('Team')['H'].shift()
hits_df_copy['AB_Next_Year'] = hits_df_copy.sort_values(['Season', 'Team'], ascending=False).groupby('Team')['AB'].shift()
hits_df_copy['HR_Next_Year'] = hits_df_copy.sort_values(['Season', 'Team'], ascending=False).groupby('Team')['HR'].shift()
hits_df_copy['RBI_Next_Year'] = hits_df_copy.sort_values(['Season', 'Team'], ascending=False).groupby('Team')['RBI'].shift()
hits_df_copy['AVG_Next_Year'] = hits_df_copy.sort_values(['Season', 'Team'], ascending=False).groupby('Team')['AVG'].shift()
hits_df_copy['SLG_Next_Year'] = hits_df_copy.sort_values(['Season', 'Team'], ascending=False).groupby('Team')['SLG'].shift()
hits_df_copy['wRC_Next_Year'] = hits_df_copy.sort_values(['Season', 'Team'], ascending=False).groupby('Team')['wRC'].shift()
hits_df_copy['WAR_Next_Year'] = hits_df_copy.sort_values(['Season', 'Team'], ascending=False).groupby('Team')['WAR'].shift()



hits_df_copy = hits_df_copy.loc[hits_df_copy['G_Next_Year'].notnull()]
hits_df_copy = hits_df_copy.loc[hits_df_copy['H_Next_Year'].notnull()]
hits_df_copy = hits_df_copy.loc[hits_df_copy['AB_Next_Year'].notnull()]
hits_df_copy = hits_df_copy.loc[hits_df_copy['HR_Next_Year'].notnull()]
hits_df_copy = hits_df_copy.loc[hits_df_copy['RBI_Next_Year'].notnull()]
hits_df_copy = hits_df_copy.loc[hits_df_copy['AVG_Next_Year'].notnull()]
hits_df_copy = hits_df_copy.loc[hits_df_copy['SLG_Next_Year'].notnull()]
hits_df_copy = hits_df_copy.loc[hits_df_copy['wRC_Next_Year'].notnull()]
hits_df_copy = hits_df_copy.loc[hits_df_copy['WAR_Next_Year'].notnull()]


hits_df_copy


In [ ]:
hits_df_copy.corr() 

In [ ]:
# splitting our data for x, y training and testing

from sklearn.model_selection import train_test_split

X = hits_df_copy[['G', 'AB', 'H', 'HR', 'RBI', 'AVG', 'SLG', 'wRC', 'WAR' ]].values
y= hits_df_copy[['G_Next_Year', 'AB_Next_Year', 'H_Next_Year', 'HR_Next_Year', 'RBI_Next_Year', 'AVG_Next_Year', 'SLG_Next_Year', 'wRC_Next_Year', 'WAR_Next_Year' ]].values

print('Original Data Shape - X: {0}, Y: {1}'.format(X.shape, y.shape))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)       #test size is 20% of original


print('Train Data Shape - X{0}, Y:{1}.'.format(X_train.shape, y_train.shape) )
print('Test Data Shape - X{0}, Y:{1}.'.format(X_test.shape, y_test.shape) )

In [ ]:
#taking our training data and fitting it into our regression model 

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


lr = LinearRegression()

lr.fit(X_train, y_train)

LinearRegression()

In [ ]:

y_pred = lr.predict(X_train)

print ('Mean number of hits:', X_train[:, 0].mean())
print ('Mean absolute error:', mean_absolute_error(y_pred, y_train))

Mean number of hits: 2364.7916666666665
Mean absolute error: 29.59835920038931


In [ ]:
hits_2019_copy = hits_2019.copy()

hits_2019_copy = hits_2019_copy.loc[:,['Season', 'Team', 'G', 'AB', 'H', 'HR', 'RBI', 'AVG', 'OBP', 'SLG', 'OPS', 'wRC', 'WAR']]

hits_2019_copy['2019_actual_g'] = hits_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['G'].shift()
hits_2019_copy['2019_actual_ab'] = hits_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['AB'].shift()
hits_2019_copy['2019_actual_hits'] = hits_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['H'].shift()
hits_2019_copy['2019_actual_hr'] = hits_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['HR'].shift()
hits_2019_copy['2019_actual_rbi'] = hits_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['RBI'].shift()
hits_2019_copy['2019_actual_avg'] = hits_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['AVG'].shift()
hits_2019_copy['2019_actual_slg'] = hits_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['SLG'].shift()
hits_2019_copy['2019_actual_wRC'] = hits_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['wRC'].shift()
hits_2019_copy['2019_actual_WAR'] = hits_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['WAR'].shift()


hits_2019_copy = hits_2019_copy.loc[hits_2019_copy['2019_actual_hits'].notnull()]

hits_2019_copy


In [ ]:
X = hits_2019_copy[['G', 'AB', 'H', 'HR', 'RBI', 'AVG', 'SLG', 'wRC', 'WAR', ]].values
y = hits_2019_copy[['2019_actual_g', '2019_actual_ab','2019_actual_hits', '2019_actual_hr', '2019_actual_rbi', '2019_actual_avg', '2019_actual_slg', '2019_actual_wRC', '2019_actual_WAR' ]].values



In [ ]:
y_pred = lr.predict(X)
print('Mean of Stats:', hits_2019_copy.mean())
print('Mean absolute error:', mean_absolute_error(y_pred, y))

Mean of Stats: Season             2018.00000
G                  2386.33333
AB                 5514.40000
H                  1367.26667
HR                 186.16667 
RBI                686.86667 
AVG                0.24783   
OBP                0.31797   
SLG                0.40923   
OPS                0.72723   
wRC                719.33333 
WAR                19.01333  
2019_actual_g      2389.46667
2019_actual_ab     5555.03333
2019_actual_hits   1401.30000
2019_actual_hr     225.86667 
2019_actual_rbi    749.03333 
2019_actual_avg    0.25217   
2019_actual_slg    0.43467   
2019_actual_wRC    780.53333 
2019_actual_WAR    18.99000  
dtype: float64
Mean absolute error: 34.894989354342385


In [ ]:
hits_2019_copy[['Predicted_G', 'Predicted_AB', 'Predicted_H', 'Predicted_HR', 'Predicted_RBI', 'Predicted_AVG', 'Predicted_SLG', 'Predicted_wRC', 'Predicted_WAR']] = y_pred

hits_2019_copy['Season'] = 2019

#hits_2019_copy = hits_2019_copy.rename(columns={'2019_actual_g':'Actual_G'})

#use sort_values to find the top predicted hits
hits_2019_copy = hits_2019_copy.loc[:,['Season','Team','2019_actual_g', '2019_actual_ab', '2019_actual_hits', '2019_actual_hr', '2019_actual_rbi' , '2019_actual_avg' , '2019_actual_slg', '2019_actual_wRC', '2019_actual_WAR', 'Predicted_G', 'Predicted_AB', 'Predicted_H', 'Predicted_HR', 'Predicted_RBI', 'Predicted_AVG', 'Predicted_SLG', 'Predicted_wRC', 'Predicted_WAR']]



In [ ]:
hits_2019_copy

In [ ]:
predicted_2020_stats = hits_2019_copy.loc[:,['Team', 'Predicted_G', 'Predicted_AB', 'Predicted_H', 'Predicted_HR', 'Predicted_RBI', 'Predicted_AVG', 'Predicted_SLG', 'Predicted_wRC', 'Predicted_WAR']]

predicted_2020_stats


In [ ]:
predicted_2020_stats.sum()

Team             BOSNYYLADCLEOAKHOUWSNCOLTBRMILCHCATLTORCINSTLTEXLAAMINSEAPITPHIARINYMCHWKCRBALDETSDPSFGMIA
Predicted_G     71491.06157                                                                                
Predicted_AB    166014.21378                                                                               
Predicted_H     41745.17895                                                                                
Predicted_HR    6329.10020                                                                                 
Predicted_RBI   21868.88340                                                                                
Predicted_AVG   7.54257                                                                                    
Predicted_SLG   12.78855                                                                                   
Predicted_wRC   22750.76427                                                                                
Predicted_WAR   595.44285   